In [ ]:
!pip install pretty_midi
!pip install gdown
!pip install miditok
!pip install midi-clip

!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth

!wget https://raw.githubusercontent.com/roostico/NesGen/refs/heads/main/utility.py

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import glob

import os
import random
import shutil
from pathlib import Path
import pretty_midi
import numpy as np
from miditok import REMI, TokenizerConfig
import json
from miditok.utils import split_files_for_training
from miditok.data_augmentation import augment_dataset
from random import shuffle
from tqdm import tqdm

import sys
import pickle
     

In [ ]:
# Get Maestro Dataset
!wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
!unzip 'maestro-v3.0.0-midi.zip'
!rm 'maestro-v3.0.0-midi.zip'
dataset_path = "/kaggle/working/maestro-v3.0.0"

In [ ]:
# Paths to the files of the dataset

midi_paths = list(Path(dataset_path).resolve().glob("**/*.mid")) + list(Path(dataset_path).resolve().glob("**/*.midi"))

midis_dir = "midis"
os.makedirs(midis_dir, exist_ok=True)


for i, midi_path in enumerate(midi_paths):
  new_midi_path = os.path.join(midis_dir, f"{i}.midi")
  shutil.move(str(midi_path), new_midi_path)


midis = list(Path("/kaggle/working/midis").resolve().glob("**/*.mid")) + list(Path("/kaggle/working/midis").resolve().glob("**/*.midi"))

def sample():
  return str(random.choice(midis))

In [ ]:
BEAT_RES = {(0, 1): 12, (1, 2): 4, (2, 4): 2, (4, 8): 1}

TOKENIZER_PARAMS = {
    "pitch_range": (21, 109),
    "beat_res": BEAT_RES,
    "num_velocities": 6,
    "special_tokens": ["BOS", "EOS"],
    "use_chords": True,
    "use_rests": True,
    "use_tempos": True,
    "num_tempos": 8,
    "tempo_range": (50, 200),  # (min_tempo, max_tempo),
}

config = TokenizerConfig(**TOKENIZER_PARAMS)

tokenizer = REMI(config)

In [ ]:
vocab_size = 1000
tokenizer.train(vocab_size=vocab_size, files_paths=midis)
processed = [Path(f"{s}") for s in midis]
print(len(processed))

valid_perc = 0.05
augment = False

total_num_files = len(processed)
num_files_valid = round(total_num_files * valid_perc)
shuffle(processed)
midi_paths_valid = processed[:num_files_valid]
midi_paths_train = processed[num_files_valid:]

# Chunk MIDIs and perform data augmentation on each subset independently

for files_paths, subset_name in (
    (midi_paths_train, "train"),
    (midi_paths_valid, "valid"),
):
    print(files_paths[0])

    # Split the MIDIs into chunks of sizes approximately about 1024 tokens

    subset_chunks_dir = Path(f"Maestro_{subset_name}")

    split_files_for_training(
        files_paths=files_paths,
        tokenizer=tokenizer,
        save_dir=subset_chunks_dir,
        max_seq_len=1024,
        num_overlap_bars=2,
    )

    # Perform data augmentation
    if augment:
        augment_dataset(
            subset_chunks_dir,
            pitch_offsets=[-12, 12],
            velocity_offsets=[-3, 3],
            duration_offsets=[-0.5, 0.5],
        )

midi_paths_train = list(Path("Maestro_train").glob("**/*.mid")) + list(Path("Maestro_train").glob("**/*.midi"))
midi_paths_valid = list(Path("Maestro_valid").glob("**/*.mid")) + list(Path("Maestro_valid").glob("**/*.midi"))

In [ ]:
def midi_valid(midi) -> bool:

    if any(ts.numerator != 4 for ts in midi.time_signature_changes):

        return False  # time signature different from 4/*, 4 beats per bar

    return True



if os.path.exists("tokenized"):

  shutil.rmtree("tokenized")


for dir in ("train", "valid"):
    tokenizer.tokenize_dataset(        
    
        Path(f"/kaggle/working/Maestro_{dir}"),
        Path(f"/kaggle/working/tokenized_{dir}"),
        midi_valid,
    
    )

In [ ]:
def read_json(path: str) -> dict:

  with open(path, "r") as f:

    return json.load(f)

def read_json_files(json_file_paths):
    """Reads a list of JSON files and returns a list of objects.
    Args:
        json_file_paths: A list of file paths to JSON files.
    Returns:
        A list of objects, where each object represents the data from a JSON file.
        Returns an empty list if any error occurs during file processing.
    """

    objects = []

    for file_path in tqdm(json_file_paths):

        try:

            objects.append(read_json(file_path))

        except FileNotFoundError:

            print(f"Error: File not found - {file_path}")

            return [] # Return empty list on error

        except json.JSONDecodeError:

            print(f"Error decoding JSON in file: {file_path}")

            return [] # Return empty list on error

    return objects

In [ ]:
tokenized_train = list(Path("tokenized_train").resolve().glob("**/*.json"))
data_objects_train = read_json_files(tokenized_train)

tokenized_valid = list(Path("tokenized_valid").resolve().glob("**/*.json"))
data_objects_valid = read_json_files(tokenized_valid)

if data_objects_train:
    print(f"\nSuccessfully read {len(data_objects_train)} training JSON files.")
else:
    print("Error reading JSON files.")

In [ ]:
encoded_train = [np.array(song["ids"][0]) for song in data_objects_train]
encoded_valid = [np.array(song["ids"][0]) for song in data_objects_valid]

In [ ]:
# tokenizer.decode([encoded_train[0][:1024]]).dump_midi("sample.mid")
all_ids_train = np.concatenate(encoded_train)
all_ids_valid = np.concatenate(encoded_valid)
import datetime
today = datetime.datetime.today()
day = today.day
month = today.month
name = "tokenizer{:d}_{:02d}{:02d}.json".format(vocab_size, month, day)
tokenizer.save(name)
np.savetxt("ids_train_{:02d}{:02d}.txt".format(month, day), all_ids_train)
np.savetxt("ids_valid_{:02d}{:02d}.txt".format(month, day), all_ids_valid)
all_ids_train = all_ids_train.astype(dtype=np.int32)
all_ids_valid = all_ids_valid.astype(dtype=np.int32)

In [ ]:
# if you need to skip all
!gdown 1FqWFCW5TjWTI8rrbBy5uwYjS3GYl9MWF # tokenizer1000_1219.json
!gdown 1Xs-5FenAaUJE_WipUDIFfQ4lFrK8VHve # ids_train_1219.txt
tokenizer = REMI(params="tokenizer1000_1219.json")
all_ids_train = np.loadtxt("ids_train_1219.txt").astype(dtype=np.int32)
#all_ids_valid = np.loadtxt("ids_valid_1217.txt").astype(dtype=np.int32)

In [ ]:
vocab_size = len(tokenizer)
seq_length = 512
normalized_seq = (all_ids_train - vocab_size / 2) / (vocab_size / 2)

# Suddivisione in sequenze
all_ids_train_seq = [normalized_seq[i:i + seq_length] 
                 for i in range(0, len(normalized_seq) - seq_length, seq_length)]

In [ ]:
class LSTMGenerator(nn.Module):
    def __init__(self, noise_dim, hidden_dim, seq_length, num_layers):
        super(LSTMGenerator, self).__init__()
        self.seq_length = seq_length
        self.hidden_dim = hidden_dim
        self.noise_dim = noise_dim
        
        # The linear layer should map the noise to (batch_size, hidden_dim * seq_length)
        self.fc = nn.Linear(noise_dim, hidden_dim * seq_length)  # Map noise to the size that works with LSTM
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.output_fc = nn.Linear(hidden_dim, 1)  # Output layer for each time step
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = x.squeeze(-1)  # Remove the last dimension -> (batch_size, noise_dim)
        x = self.fc(x)  # Map noise to (batch_size, hidden_dim * seq_length)
        x = x.view(-1, self.seq_length, self.hidden_dim)  # Reshape to (batch_size, seq_length, hidden_dim)
        out, _ = self.lstm(x)  # Pass through LSTM
        out = self.output_fc(out)  # Map hidden states to output dimension
        return self.tanh(out)  # Shape: (batch_size, seq_length, 1)

class LSTMDiscriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(LSTMDiscriminator, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)  # Predict a single scalar for real/fake
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.lstm(x)  # Pass through LSTM
        out = out[:, -1, :]  # Use the last time step's hidden state: (batch_size, hidden_dim)
        out = self.fc(out)  # Map to a single value: (batch_size, 1) 
        return out  # Real/fake probability

class GAN:
    def __init__(self, 
                 generator_builder, 
                 discriminator_builder, 
                 noise_dim,
                 seq_length,
                 generator_layers,
                 discriminator_layers,
                 generator_hidden_dim,
                 discriminator_hidden_dim,
                 loss = nn.BCEWithLogitsLoss(),
                ):
        self.G = generator_builder(
            noise_dim  = noise_dim,
            hidden_dim = generator_hidden_dim,
            seq_length = seq_length,
            num_layers = generator_layers
        )
        self.D = discriminator_builder(
            input_dim  = 1,
            hidden_dim = discriminator_hidden_dim,
            num_layers = discriminator_layers,
        )
        self.noise_dim = noise_dim
        self.seq_length = seq_length
        self.loss = loss
        self.optimizer_G = optim.Adam(self.G.parameters(), lr=0.0002)
        self.optimizer_D = optim.Adam(self.D.parameters(), lr=0.0002)

    def train(
        self, 
        dataloader, 
        epochs, 
        device, 
        warmup_epochs=0, 
        g_steps=0, 
        steps_each_print=5
    ):
        print(f"Starting training with {epochs} epochs")
        self.G = self.G.to(device)
        self.D = self.D.to(device)
        
        iteration_count = len(dataloader)  # Number of batches per epoch
        batch_size = dataloader.batch_size
        
        for epoch in range(epochs):
            pbar = tqdm(total=iteration_count, position=0, leave=True)
            for step, real_data in enumerate(dataloader):
                batch_size = real_data.size(0)
                real_data = real_data.to(device).unsqueeze(-1)  # Shape: (batch_size, seq_length, 1)
                
                # Labels
                real_labels = torch.ones(batch_size, 1).to(device)
                fake_labels = torch.zeros(batch_size, 1).to(device)
        
                if epoch < warmup_epochs:  # Train Generator more during initial epochs
                    for _ in range(g_steps):
                        self.G.zero_grad()
                        z = torch.randn(batch_size, noise_dim, 1).to(device)
                        fake_data = self.G(z)
                        outputs = self.D(fake_data)
                        loss_G = self.loss(outputs, real_labels)
                        loss_G.backward()
                        self.optimizer_G.step()
                
                # Train Discriminator
                self.D.zero_grad()
                z = torch.randn(batch_size, self.noise_dim).to(device)  # Correct noise shape
                fake_data = self.G(z)
        
                outputs_real = self.D(real_data)  # Pass real_data through Discriminator
                loss_real = self.loss(outputs_real, real_labels)
        
                outputs_fake = self.D(fake_data)  # Pass fake_data through Discriminator
                loss_fake = self.loss(outputs_fake, fake_labels)
        
                loss_D = loss_real + loss_fake
                loss_D.backward()
                self.optimizer_D.step()
        
                # Train Generator
                self.G.zero_grad()
                z = torch.randn(batch_size, noise_dim).to(device)
                fake_data = self.G(z)
        
                outputs = self.D(fake_data)
                loss_G = self.loss(outputs, real_labels)  # Generator wants to fool the Discriminator
                loss_G.backward()
                self.optimizer_G.step()
                if (step + 1) % steps_each_print == 0:
                    pbar.set_description(
                        f"D Loss: {loss_D.item():.4f}, " +
                        f"G Loss: {loss_G.item():.4f}"
                    )
                    pbar.update(steps_each_print)
            pbar.n = pbar.total  
            pbar.refresh()    
            pbar.close()
        
            print(f'Epoch [{epoch+1}/{epochs}] Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}')

    def generate(self, n, tokenizer, base_output_name, device):
        z = torch.randn(n, self.noise_dim, 1)  # Rumore casuale
        z = z.to(device)
        generated_data = self.G(z).cpu().detach().numpy()

        boundary = int(len(tokenizer) / 2)
        predictions = [x * boundary + boundary for x in generated_data]
        for i in range(len(predictions)):
            pred = predictions[i]                                # [[5.0], [6.0], [7.0]]
            pred_tokens = np.concatenate(pred).astype(np.int32)  # [5, 6, 7]
            
            decoded = tokenizer.decode([pred_tokens])
            decoded.dump_midi(f"{base_output_name}_{i}.mid")

In [ ]:
# Parametri
generator_dim = 256
discriminator_dim = generator_dim // 2
num_layers = 2
batch_size = 128
noise_dim = 256

gan = GAN(
    LSTMGenerator,
    LSTMDiscriminator,
    noise_dim                = noise_dim,
    seq_length               = seq_length,
    generator_layers         = num_layers,
    discriminator_layers     = num_layers,
    generator_hidden_dim     = generator_dim,
    discriminator_hidden_dim = discriminator_dim,
)

# Training

In [ ]:
# Dataset personalizzato
class TokenDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32)

dataset = TokenDataset(all_ids_train_seq)
dataloader = DataLoader(dataset, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training loop
epochs = 1
warmup_epochs = 3
g_steps = 0
steps_each_print = 5

gan.train(
    dataloader       = dataloader, 
    epochs           = epochs, 
    device           = device, 
    warmup_epochs    = warmup_epochs, 
    g_steps          = g_steps, 
    steps_each_print = steps_each_print
)

In [ ]:
n_generations = 5
base_name = "generated"

gan.generate(
    n                 = n_generations,
    tokenizer         = tokenizer, 
    base_output_name  = base_name, 
    device            = device,
)

In [ ]:
from IPython import display

_SAMPLING_RATE = 16000

def display_audio(file, seconds=10):
  pm = pretty_midi.PrettyMIDI(file)
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

display_audio(f"{base_name}_0.mid")

In [ ]:
from IPython.display import FileLink
FileLink(r'generated.mid')